In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

In [0]:
char = ['c','h','o','p','r','t','y']

In [0]:
x_data = [[2,7,14,15,17,19,24]]
x_one_hot = [[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
            ]

In [0]:
y_data = [15,24,19,14,17,2,7]

In [0]:
inputs = Variable(torch.Tensor(x_one_hot))
labels = Variable(torch.LongTensor(y_data))

In [0]:
num_classes = 7
input_size = 7 
hidden_size = 7
batch_size = 1 # One Sentence
sequence_length = 1 # Let go one by one 
num_layers = 1

In [0]:
class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                          batch_first=True)
    
    def forward(self, x, hidden, cell):
        x = x.view(batch_size, sequence_length, input_size)
        out, (hidden, cell) = self.lstm(x, hidden, cell)
        out= out.view(-1, num_classes)
        return hidden,out, cell

    def init_hidden(self):
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))

In [0]:
model = LSTMModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [0]:
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()
    cell = model.init_hidden()
    print("Predicted String")
    for ins, label in zip(inputs,labels):
        hidden, output, cell = model(ins, hidden, cell)
        val, idx = output.max(1)
        print(char[idx.data[0]])
        loss+=criterion(output, label)
    
    print(", epoch: %d, loss: %1.3f" % (epoch+1, loss.data[0]))

    loss.backward()
    optimizer.step()